# 豆瓣爬虫

In [48]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
headers = {'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1"}
# UTF-8 is backwards compatible with ASCII. UTF-8 is the preferred encoding for e-mail and web pages
# res.encoding = 'utf-8'

In [6]:
movie = input('Pls input website address that you want to fetch')
table = input('Pls input the file name that you want to save to')
path = './'+table+'.txt'
file = open(path,'w')

Pls input website address that you want to fetchhttps://movie.douban.com/subject/34841067/
Pls input the file name that you want to save totest


## 写入txt文件

In [9]:
# 写入txt文件
def getTxtFile(url):
    res = requests.get(url,headers=headers)
    soup = BeautifulSoup(res.text,'html.parser')
    
    #comments = soup.findALL('div', class_='comment-item')
    comments = soup.findAll('div', class_='comment-item')
    for comment in comments:
        # 三元表达式
        username = comment.find('a')['title'] and comment.find('a')['title'] or ''
        # rating = comment.find('span',{'class':'rating'})['title']
        span = comment.find('span',class_='rating')
        # 有一页的span为空，会报错
        if span == None:
            continue
        rating = comment.find('span',class_='rating')['title'] or ''
        # s.strip(x):删除s字符串中开头、结尾处，位于x删除序列的字符
        # time = comment.find('span',class_='comment-time').text.split()
        # split是分割函数，将字符串分割成“字符”，保存在一个列表中。默认不带参数为空格分割
        time = comment.find('span',class_='comment-time')['title'].split()[0] and comment.find('span',class_='comment-time')['title'].split()[0] or ''
        content = comment.find('span',class_='short').text and comment.find('span',class_='short').text or ''
        vote = comment.find('span',class_='votes').text and comment.find('span',class_='votes').text or ''
        data = username + ';' + rating + ';' + time + ';' + content + ';' + vote + '\n'
        try:
            file.write(data)
        except Exception:
            None

In [ ]:
for i in range(10):
    num = i*20
    # https://movie.douban.com/subject/27092785/
    Url = movie + '/comments?start='+str(num)+'&limit=20&sort=new_score&status=P'
    print(Url)
    getTxtFile(Url)

file.close()
print('The data has been fetched.')

## 写入dataframe

In [55]:
# 写入dataframe
username_list = []
rating_list = []
time_list = []
content_list = []
vote_list = []
def getDataFrame(url):
    res = requests.get(url,headers=headers)
    soup = BeautifulSoup(res.text,'html.parser')
    comments = soup.findAll('div', class_='comment-item')
    for comment in comments:
        try:
            # 三元表达式的语法糖：expression?result1:result2
            username = comment.find('a')['title'] and comment.find('a')['title'] or ''
            # rating = comment.find('span',{'class':'rating'})['title']
            span = comment.find('span',class_='rating') and comment.find('span',class_='rating') or ''
    #         if span == None:
    #             continue
            rating = comment.find('span',class_='rating')['title'] and comment.find('span',class_='rating')['title'] or ''
            time = comment.find('span',class_='comment-time')['title'].split()[0] and comment.find('span',class_='comment-time')['title'].split()[0] or ''
            content = comment.find('span',class_='short').text and comment.find('span',class_='short').text or ''
            vote = comment.find('span',class_='votes').text and comment.find('span',class_='votes').text or ''
            #print(username,rating,time,content,vote)
            username_list.append(username)
            rating_list.append(rating)
            time_list.append(time)
            content_list.append(content)
            vote_list.append(vote)
        except Exception:
            continue

In [56]:
username_list = []
span_list = []
rating_list = []
time_list = []
content_list = []
vote_list = []
for i in range(1,11,1):
    num = i*20
    # https://movie.douban.com/subject/27092785/
    Url = movie + '/comments?start='+str(num)+'&limit=20&sort=new_score&status=P'
    getDataFrame(Url)

In [54]:
data = pd.DataFrame({'username':username_list,'rating':rating_list,'time':time_list,'content':content_list,'vote':vote_list})
data.head()

,username,rating,time,content,vote
0,北投少年他,较差,2021-02-12,把二十多分钟的小品拉长到两个小时，没有好导演和好剧本还是省省吧。,1390
1,想睡觉,推荐,2021-02-13,你以为你已经很爱很爱妈妈了，但妈妈远比你想象中更爱更爱更爱你,2093
2,蜜汁,力荐,2021-02-12,意识到妈妈什么都知道的时候，我哭到抽搐。贾玲太有才了。在读你和我时就在想，逝去的父母是靠儿女...,5049
3,豆川,还行,2021-02-13,前一个小时强行塞包袱，后面干嚎占二十分钟。。。,1053
4,私享史,还行,2021-02-12,7分。豆瓣8.2分我是真觉得过分了，就电影论电影的话，贾玲这部作品最大的问题在于它不太像电影...,818


# 可视化分析